# BAIS:3250 Data Wrangling
## Final Project: Web Scraping Notebook
## Sylvia Jerden 5/9

In [2]:
# importing necessary packages
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException

from webdriver_manager.chrome import ChromeDriverManager

import time
import random

In [2]:
# specific url I want to scrape from
# all make and models of used cars within a 250 mile radius of Iowa City (52242)
url = "https://www.cars.com/shopping/results/?stock_type=used&makes%5B%5D=&models%5B%5D=&maximum_distance=250&zip=52242"

In [3]:
# opening bot operated chrome tab
browser = webdriver.Chrome()
browser.maximize_window()
browser.get(url)

In [4]:
# starting a blank list for just the urls
# we'll loop through and gather the urls first, then visit each url individually to scrape the data from the page
urls = []

In [5]:
# building a random scroll function, as to avoid detection
def random_scroll(browser, total_scroll_time):
    total_height = browser.execute_script("return document.body.scrollHeight")
    scroll_steps = random.randint(3, 10)
    scroll_position = 0
    for step in range(scroll_steps):
        scroll_position += total_height // scroll_steps
        browser.execute_script(f"window.scrollTo(0, {scroll_position});")
        time.sleep(random.uniform(1, 4))
        new_height = browser.execute_script("return document.body.scrollHeight")
        if new_height > total_height:
            total_height = new_height

In [6]:
urls = set()  # Set to store unique links
max_pages = 46  # Adjust as needed
page_count = 0

try:
    close_button = WebDriverWait(browser, 5).until(
        EC.element_to_be_clickable((By.CLASS_NAME, 'onetrust-close-btn-handler'))
    )
    close_button.click()  
    print("Cookie popup closed.") # closing the cookies popup, as it prevents selenium from progressing to the next page
    time.sleep(2)  # allow page time to refresh
except:
    print("No cookie popup detected, continuing...")


while page_count < max_pages:
    time.sleep(2)  # Ensure page fully loads

    # Scrape vehicle links
    link_elements = browser.find_elements(By.CLASS_NAME, 'vehicle-card-link')
    
    for link in link_elements:  # Skip sponsored posts
        urls.add(link.get_attribute('href')) 
    try:
        next_button = WebDriverWait(browser, 5).until(
            EC.element_to_be_clickable((By.ID, "next_paginate"))
        )
        browser.execute_script("arguments[0].scrollIntoView();", next_button) 
        next_button.click()
        page_count += 1 
        time.sleep(3) 
    except:
        print("Next button not found or unable to click.") # breaking in case the next page button cannot be found
        break 


urls = sorted(urls)

# Filter out sponsored links based on their identifier(s)
filtered_urls = [link for link in urls if not link.endswith(("?attribution_type=isa", "?attribution_type=premier"))]

for index, link in enumerate(filtered_urls):
    print(f"Link {index + 1}: {link}")

Cookie popup closed.
Link 1: https://www.cars.com/vehicledetail/0023a179-4416-4af6-b001-a43cf19e21c3/
Link 2: https://www.cars.com/vehicledetail/0036339e-627a-4bda-a0ea-df19667a6499/
Link 3: https://www.cars.com/vehicledetail/005dd6bb-de56-4b94-a24b-4eae7493504e/
Link 4: https://www.cars.com/vehicledetail/00907437-a070-4df1-93c1-6b141cb96788/
Link 5: https://www.cars.com/vehicledetail/00f14b87-0f28-4302-a0fc-e95e36bbd46e/
Link 6: https://www.cars.com/vehicledetail/013fed5a-68c0-427c-895e-53b43bffdd55/
Link 7: https://www.cars.com/vehicledetail/0177cf31-a9a0-442a-b5a4-64bcdb9fd0e8/
Link 8: https://www.cars.com/vehicledetail/01d7c472-c854-446b-9731-c64e164565dd/
Link 9: https://www.cars.com/vehicledetail/01eae64a-aa3e-4cff-96aa-62b82f24ac3f/
Link 10: https://www.cars.com/vehicledetail/020ec8ad-6044-4398-b2c1-b81b3686882f/
Link 11: https://www.cars.com/vehicledetail/0225ef06-3f76-41af-aa7f-f463e8a42fe3/
Link 12: https://www.cars.com/vehicledetail/022f03df-1968-4095-ae81-fff5ed289199/
Link

In [7]:
# creating blank lists to fill with scraped data

# the data frame I plan to vertically merge with has year, make, and model as a seperate section. I will seperate them mannually later, as the listing 
# title is one entry, and I cannot scrape them seperately
other_lists = {
    'name': [],
    'price': []
}

In [8]:
    # Define blank lists for each type of data
data_lists = {
    'Exterior color': [],
    'Interior color': [],
    'Drivetrain': [],
    'MPG': [],
    'Fuel type': [],
    'Transmission': [],
    'Engine': [],
    'VIN': [],
    'Stock #': [],
    'Mileage': []
}

In [9]:
expected_rows = ['Exterior color', 'Interior color', 'Drivetrain', 'MPG', 'Fuel type', 'Transmission', 'Engine', 'VIN', 'Stock #', 'Mileage']

In [10]:
entry_count = 0
for url in urls:
 #   if entry_count >= 10:
  #      break 
   # else: (stops after certain entries, good for troubleshooting if needed) 
    if entry_count >= len(urls): 
        break
    entry_count += 1
    print('-'*70)
    print(f"Visiting {url}")
    print(f"Page count: {entry_count}")
    print('-'*70)
    time.sleep(3)
    browser.get(url)
    random_scroll(browser, 3)

    # Scraping for car name 
    title_element = browser.find_element(By.CLASS_NAME, "listing-title")
    car_name = title_element.text
    print(f"{car_name}")
    other_lists['name'].append(car_name)

    # Scraping price
    price_element = browser.find_element(By.CLASS_NAME, "primary-price")
    prices = price_element.text
    print(f"{prices}")
    other_lists['price'].append(prices)
    
    # Scrape other data to data_lists (might filter this further later, as not all the data is necessary)
    data = {}
    section = WebDriverWait(browser, 5).until(
        EC.presence_of_element_located((By.XPATH, '//section[contains(@class, "sds-page-section basics-section")]'))
    )
    dts = section.find_elements(By.TAG_NAME, 'dt')
    dds = section.find_elements(By.TAG_NAME, 'dd')

    for dt, dd in zip(dts, dds):
        data[dt.text.strip()] = dd.text.strip()  # populates dictionary
    print("Scraped Data:", data)

    for row in expected_rows:
        if row not in data:
            data[row] = "-"  # assigns default value

    # Populate the lists
    for dt, dd in data.items():
        if dt in data_lists:
            if dd == "-" or not dd.strip():
                dd = np.nan
            data_lists[dt].append(dd)
#            print(f"Added {dd} to {dt}")  # Debugging (remove)

    # Debugging
#    print("Data Lists:")
#    for key, values in data_lists.items():
#        print(f"{key}: {values}")

----------------------------------------------------------------------
Visiting https://www.cars.com/vehicledetail/0023a179-4416-4af6-b001-a43cf19e21c3/
Entry count: 1
----------------------------------------------------------------------
2018 Nissan Altima 2.5 SR
$8,999
Scraped Data: {'Exterior color': 'Gun Metallic', 'Interior color': 'Sport Interior', 'Drivetrain': 'Front-wheel Drive', 'MPG': '26–37', 'Fuel type': 'Gasoline', 'Transmission': 'CVT with Xtronic', 'Engine': '2.5L I-4 DOHC, variable valve control, regular unleaded, engine', 'VIN': '1N4AL3AP6JC131884', 'Stock #': 'J1919C', 'Mileage': '135,065 mi.'}
----------------------------------------------------------------------
Visiting https://www.cars.com/vehicledetail/0036339e-627a-4bda-a0ea-df19667a6499/
Entry count: 2
----------------------------------------------------------------------
2023 Ford F-150 XLT
$41,777
Scraped Data: {'Exterior color': 'Agate Black Metallic', 'Interior color': 'Black', 'Drivetrain': 'Four-wheel Dr

InvalidSessionIdException: Message: invalid session id: session deleted as the browser has closed the connection
from disconnected: not connected to DevTools
  (Session info: chrome=136.0.7103.93)
Stacktrace:
	GetHandleVerifier [0x00007FF6AC62CF25+75717]
	GetHandleVerifier [0x00007FF6AC62CF80+75808]
	(No symbol) [0x00007FF6AC3F8F9A]
	(No symbol) [0x00007FF6AC3E4E35]
	(No symbol) [0x00007FF6AC409DB4]
	(No symbol) [0x00007FF6AC47EE75]
	(No symbol) [0x00007FF6AC49ECC2]
	(No symbol) [0x00007FF6AC477153]
	(No symbol) [0x00007FF6AC440421]
	(No symbol) [0x00007FF6AC4411B3]
	GetHandleVerifier [0x00007FF6AC92D6FD+3223453]
	GetHandleVerifier [0x00007FF6AC927CA2+3200322]
	GetHandleVerifier [0x00007FF6AC945AD3+3322739]
	GetHandleVerifier [0x00007FF6AC6469FA+180890]
	GetHandleVerifier [0x00007FF6AC64E0FF+211359]
	GetHandleVerifier [0x00007FF6AC635274+109332]
	GetHandleVerifier [0x00007FF6AC635422+109762]
	GetHandleVerifier [0x00007FF6AC61BA39+4825]
	BaseThreadInitThunk [0x00007FFE0172E8D7+23]
	RtlU

For scraping my data, my goal was around 1000 rows of data. However, the furthest I got was 281 rows. When trying to run it again, at around 250 rows it would stop collecting the price altogether. While not ideal, I do believe that this is enough data to perform my analysis. 

In [ ]:
# merging my two dictionaries together
merged = other_lists | data_lists 

In [ ]:
merged

In [ ]:
# converting the dictionaries into a dataframe
cars_df = pd.DataFrame(merged)

In [ ]:
cars_df

In [ ]:
# saving to a csv so if I need to redo anything, I don't have to scrape again
cars_df.to_csv("cars_df2.csv")

In [27]:
cars_df2 = pd.read_csv("cars_df2.csv")

In [28]:
cars_df2

,Unnamed: 0,name,price,Exterior color,Interior color,Drivetrain,MPG,Fuel type,Transmission,Engine,VIN,Stock #,Mileage
0,0,NaN,"$8,999",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2023 Ford F-150 XLT,"$41,777",Agate Black Metallic,Black,Four-wheel Drive,17–22,Gasoline,Automatic,"EcoBoost 2.7L V-6 port/direct injection, DOHC,...",1FTEW1EP1PFB93427,PG8051,"36,670 mi."
2,2,2022 Subaru Outback Wilderness,"$26,557",Crystal Black Silica,Gray,All-wheel Drive,22–26,Gasoline,CVT Lineartronic,"2.4L H-4 gasoline direct injection, DOHC, vari...",4S4BTGUD9N3141835,N3141835,"77,560 mi."
3,3,2022 Mitsubishi Outlander SEL 2.5 2WD,"$27,500",Alloy Silver Metallic,Black,Front-wheel Drive,24–31,Gasoline,CVT,"2.5L I-4 gasoline direct injection, DOHC, vari...",JA4J3VA85NZ086447,38134,"21,957 mi."
4,4,2019 Kia Sorento LX,"$20,499",Silver,Satin Black,All-wheel Drive,21–26,Gasoline,Automatic,I4,5XYPGDA38KG433024,8H1638T,"27,008 mi."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,275,2023 Toyota Tacoma Base,"$47,975",Black,Black/Gun Metal,–,18–22,Gasoline,Automatic,Regular Unleaded V-6 3.5 L/211,3TMDZ5BN1PM159940,T559940,"18,383 mi."
276,276,2022 Jeep Grand Cherokee Laredo,"$27,789",Diamond Black Crystal Pearlcoat,Black,Four-wheel Drive,18–25,Gasoline,Automatic,"Pentastar 3.6L V-6 DOHC, variable valve contro...",1C4RJFAG8NC112716,P11304,"41,805 mi."
277,277,2022 Chevrolet Silverado 1500 LTZ,"$46,995",Summit White,Jet Black,Four-wheel Drive,20–26,Diesel,Automatic,"Duramax 3L I-6 diesel direct injection, DOHC, ...",1GCUDGET5NZ601330,B1146,"38,741 mi."
278,278,2022 Chevrolet Silverado 1500 ZR2,"$46,880",Dark Ash Metallic,Jet Black,Four-wheel Drive,14–17,Gasoline,10-Speed Automatic,6.2L V8 16V GDI OHV,3GCUDHEL4NG676476,P294331,"76,625 mi."


In [30]:
# Splitting name into three categories
cars_df2[['model_year', 'brand', 'model']] = cars_df2['name'].str.split(' ',n=2, expand=True)

In [31]:
cars_df2

,Unnamed: 0,name,price,Exterior color,Interior color,Drivetrain,MPG,Fuel type,Transmission,Engine,VIN,Stock #,Mileage,model_year,brand,model
0,0,NaN,"$8,999",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2023 Ford F-150 XLT,"$41,777",Agate Black Metallic,Black,Four-wheel Drive,17–22,Gasoline,Automatic,"EcoBoost 2.7L V-6 port/direct injection, DOHC,...",1FTEW1EP1PFB93427,PG8051,"36,670 mi.",2023,Ford,F-150 XLT
2,2,2022 Subaru Outback Wilderness,"$26,557",Crystal Black Silica,Gray,All-wheel Drive,22–26,Gasoline,CVT Lineartronic,"2.4L H-4 gasoline direct injection, DOHC, vari...",4S4BTGUD9N3141835,N3141835,"77,560 mi.",2022,Subaru,Outback Wilderness
3,3,2022 Mitsubishi Outlander SEL 2.5 2WD,"$27,500",Alloy Silver Metallic,Black,Front-wheel Drive,24–31,Gasoline,CVT,"2.5L I-4 gasoline direct injection, DOHC, vari...",JA4J3VA85NZ086447,38134,"21,957 mi.",2022,Mitsubishi,Outlander SEL 2.5 2WD
4,4,2019 Kia Sorento LX,"$20,499",Silver,Satin Black,All-wheel Drive,21–26,Gasoline,Automatic,I4,5XYPGDA38KG433024,8H1638T,"27,008 mi.",2019,Kia,Sorento LX
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,275,2023 Toyota Tacoma Base,"$47,975",Black,Black/Gun Metal,–,18–22,Gasoline,Automatic,Regular Unleaded V-6 3.5 L/211,3TMDZ5BN1PM159940,T559940,"18,383 mi.",2023,Toyota,Tacoma Base
276,276,2022 Jeep Grand Cherokee Laredo,"$27,789",Diamond Black Crystal Pearlcoat,Black,Four-wheel Drive,18–25,Gasoline,Automatic,"Pentastar 3.6L V-6 DOHC, variable valve contro...",1C4RJFAG8NC112716,P11304,"41,805 mi.",2022,Jeep,Grand Cherokee Laredo
277,277,2022 Chevrolet Silverado 1500 LTZ,"$46,995",Summit White,Jet Black,Four-wheel Drive,20–26,Diesel,Automatic,"Duramax 3L I-6 diesel direct injection, DOHC, ...",1GCUDGET5NZ601330,B1146,"38,741 mi.",2022,Chevrolet,Silverado 1500 LTZ
278,278,2022 Chevrolet Silverado 1500 ZR2,"$46,880",Dark Ash Metallic,Jet Black,Four-wheel Drive,14–17,Gasoline,10-Speed Automatic,6.2L V8 16V GDI OHV,3GCUDHEL4NG676476,P294331,"76,625 mi.",2022,Chevrolet,Silverado 1500 ZR2


In [32]:
# renaming columns to match the case and rules of the other dataframe
cars_df2 = cars_df2.rename(columns={
    'Exterior color': 'ext_col',
    'Interior color': 'int_col',
    'Fuel type': 'fuel_type',
    'Transmission': 'transmission', 
    'Mileage': 'mileage',
    'Engine': 'engine'
})

In [33]:
cars_df2

,Unnamed: 0,name,price,ext_col,int_col,Drivetrain,MPG,fuel_type,transmission,engine,VIN,Stock #,mileage,model_year,brand,model
0,0,NaN,"$8,999",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2023 Ford F-150 XLT,"$41,777",Agate Black Metallic,Black,Four-wheel Drive,17–22,Gasoline,Automatic,"EcoBoost 2.7L V-6 port/direct injection, DOHC,...",1FTEW1EP1PFB93427,PG8051,"36,670 mi.",2023,Ford,F-150 XLT
2,2,2022 Subaru Outback Wilderness,"$26,557",Crystal Black Silica,Gray,All-wheel Drive,22–26,Gasoline,CVT Lineartronic,"2.4L H-4 gasoline direct injection, DOHC, vari...",4S4BTGUD9N3141835,N3141835,"77,560 mi.",2022,Subaru,Outback Wilderness
3,3,2022 Mitsubishi Outlander SEL 2.5 2WD,"$27,500",Alloy Silver Metallic,Black,Front-wheel Drive,24–31,Gasoline,CVT,"2.5L I-4 gasoline direct injection, DOHC, vari...",JA4J3VA85NZ086447,38134,"21,957 mi.",2022,Mitsubishi,Outlander SEL 2.5 2WD
4,4,2019 Kia Sorento LX,"$20,499",Silver,Satin Black,All-wheel Drive,21–26,Gasoline,Automatic,I4,5XYPGDA38KG433024,8H1638T,"27,008 mi.",2019,Kia,Sorento LX
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,275,2023 Toyota Tacoma Base,"$47,975",Black,Black/Gun Metal,–,18–22,Gasoline,Automatic,Regular Unleaded V-6 3.5 L/211,3TMDZ5BN1PM159940,T559940,"18,383 mi.",2023,Toyota,Tacoma Base
276,276,2022 Jeep Grand Cherokee Laredo,"$27,789",Diamond Black Crystal Pearlcoat,Black,Four-wheel Drive,18–25,Gasoline,Automatic,"Pentastar 3.6L V-6 DOHC, variable valve contro...",1C4RJFAG8NC112716,P11304,"41,805 mi.",2022,Jeep,Grand Cherokee Laredo
277,277,2022 Chevrolet Silverado 1500 LTZ,"$46,995",Summit White,Jet Black,Four-wheel Drive,20–26,Diesel,Automatic,"Duramax 3L I-6 diesel direct injection, DOHC, ...",1GCUDGET5NZ601330,B1146,"38,741 mi.",2022,Chevrolet,Silverado 1500 LTZ
278,278,2022 Chevrolet Silverado 1500 ZR2,"$46,880",Dark Ash Metallic,Jet Black,Four-wheel Drive,14–17,Gasoline,10-Speed Automatic,6.2L V8 16V GDI OHV,3GCUDHEL4NG676476,P294331,"76,625 mi.",2022,Chevrolet,Silverado 1500 ZR2


### Changing datatypes to be able to do calculations later

In [40]:
cars_df2['price'] = cars_df2['price'].str.replace("$","").str.replace(",","")
cars_df2['price'] = cars_df2['price'].astype("int")

In [34]:
cars_df2 = cars_df2.iloc[1:].copy()

In [38]:
cars_df2 = cars_df2.dropna(subset=['mileage'])  # Drop rows where 'mileage' is missing
cars_df2['mileage'] = cars_df2['mileage'].str.replace("mi.","").str.replace(",","")
cars_df2['mileage'] = cars_df2['mileage'].astype("int")

In [45]:
cars_df2.drop(['name', 'Stock #', 'VIN', 'Unnamed: 0'], axis=1, inplace=True)

KeyError: "['name', 'Stock #', 'VIN', 'Unnamed: 0'] not found in axis"

In [46]:
cars_df2.drop(['engine'], axis=1, inplace=True)

In [48]:
new_order = ['price','mileage','model_year','brand','model','ext_col','int_col','fuel_type','Drivetrain','MPG','transmission']
cars_df2 = cars_df2[new_order]

In [51]:
cars_df2['data_year'] = 2025

C:\Users\sylvi\AppData\Local\Temp\ipykernel_72460\2459806397.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cars_df2['data_year'] = 2025


In [52]:
cars_df2

,price,mileage,model_year,brand,model,ext_col,int_col,fuel_type,Drivetrain,MPG,transmission,data_year
1,41777,36670,2023,Ford,F-150 XLT,Agate Black Metallic,Black,Gasoline,Four-wheel Drive,17–22,Automatic,2025
2,26557,77560,2022,Subaru,Outback Wilderness,Crystal Black Silica,Gray,Gasoline,All-wheel Drive,22–26,CVT Lineartronic,2025
3,27500,21957,2022,Mitsubishi,Outlander SEL 2.5 2WD,Alloy Silver Metallic,Black,Gasoline,Front-wheel Drive,24–31,CVT,2025
4,20499,27008,2019,Kia,Sorento LX,Silver,Satin Black,Gasoline,All-wheel Drive,21–26,Automatic,2025
5,37998,46140,2019,Porsche,Cayenne,Blue,BLACK,Gasoline,Four-wheel Drive,19–23,Automatic,2025
...,...,...,...,...,...,...,...,...,...,...,...,...
275,47975,18383,2023,Toyota,Tacoma Base,Black,Black/Gun Metal,Gasoline,–,18–22,Automatic,2025
276,27789,41805,2022,Jeep,Grand Cherokee Laredo,Diamond Black Crystal Pearlcoat,Black,Gasoline,Four-wheel Drive,18–25,Automatic,2025
277,46995,38741,2022,Chevrolet,Silverado 1500 LTZ,Summit White,Jet Black,Diesel,Four-wheel Drive,20–26,Automatic,2025
278,46880,76625,2022,Chevrolet,Silverado 1500 ZR2,Dark Ash Metallic,Jet Black,Gasoline,Four-wheel Drive,14–17,10-Speed Automatic,2025


In [53]:
cars_df2.to_csv("cars_scraped_cleaned.csv")